# Домашнее задание 3 [10 баллов]


# До 27.03.18 23:59

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная

### Предварительная обработка данных [2 балла]

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [1]:
# прочитаем файлы
female = open('female.txt', 'r').read().split('\n')
male = open('male.txt', 'r').read().split('\n')
print(len(female), len(male))

5001 2943


#### Удаляем неоднозначные имена

In [2]:
# найдем имена, которые одинаковы для двух списков
result = set(female) & set(male)

In [3]:
# удалим их!
female = list(set(female) - result)
male = list(set(male) - result)

In [4]:
print(len(female), len(male))

4636 2578


In [5]:
print(male[:100])

['Rodge', 'Rollin', 'Izzy', 'Trenton', 'Samuel', 'Gasper', 'Solomon', 'Corby', 'Rabbi', 'Zebedee', 'Agustin', 'Bjorn', 'Bogart', 'Traver', 'Raymundo', 'Gerhardt', 'Diego', 'Hamlet', 'Warde', 'Rockwell', 'Reuben', 'Napoleon', 'Bernd', 'Beck', 'Torrin', 'Hezekiah', 'Templeton', 'Maxwell', 'Benson', 'Isaak', 'Izaak', 'Rolph', 'Sol', 'Rutger', 'Rodrigo', 'Salomon', 'Simeon', 'Broddie', 'Rik', 'Ingemar', 'Ingelbert', 'Harrison', 'Rolando', 'Lincoln', 'Forrest', 'Dieter', 'Clarke', 'Ashby', 'Chauncey', 'Nick', 'Ritch', 'Salvidor', 'Niels', 'Osbourn', 'Paddie', 'Sergeant', 'Haywood', 'Kendrick', 'Mike', 'Wash', 'Corwin', 'Wallache', 'Elwood', 'Hiro', 'Jorge', 'Stirling', 'Haskel', 'Job', 'Brody', 'Aristotle', 'Anatole', 'Bubba', 'Carter', 'Jerrome', 'Jeffrey', 'Ulberto', 'Harland', 'Roice', 'Redmond', 'Zacharia', 'Nickey', 'Kevan', 'Antoine', 'Armando', 'Che', 'Sloan', 'Herrick', 'Simmonds', 'Conroy', 'David', 'Townie', 'Gerard', 'Marwin', 'Matthew', 'Zary', 'Herschel', 'Keenan', 'Shurlocke',

#### Балансируем выборки

In [6]:
# oversampling!
missing = len(female) - len(male)
male += male[:missing]

In [7]:
# shuffling
import random
random.shuffle(male)

In [8]:
print(len(female), len(male))

4636 4636


In [9]:
boundary = int(len(female) * 0.8)
x_train = female[:boundary] + male[:boundary]
x_test = female[boundary:] + male[boundary:]
y_train = ['female'] * (boundary ) + ['male'] * (boundary)
y_test = ['female'] * (len(x_test)//2) + ['male'] * (len(x_test)//2)
print(len(x_train), len(x_test), len(y_train), len(y_test))

7416 1856 7416 1856


In [10]:
import pandas as pd
import numpy as np
index = random.sample(range(len(x_train)), len(x_train))
df = pd.DataFrame({'name': x_train, 'class': y_train})
df.head()

,class,name
0,female,Benoite
1,female,Filide
2,female,Gretal
3,female,Shauna
4,female,Elwira


In [11]:
# перемешаем
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,class,name
0,female,Josie
1,male,Barris
2,female,Tamarah
3,male,Teador
4,male,Salomone


##  Часть 2. Базовая 

### Базовый метод классификации [3 балла]

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

In [67]:
# from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [46]:
# def get_ngrams(n):
#     return [list(ngrams(name, n)) for name in df.name]
# df['bigrams'] = [list(''.join(ng) for ng in ngrams(name, 2)) for name in df.name]
# df['trigrams'] = [list(ngrams(name, 3)) for name in df.name]
# df['quadrigrams'] = [list(ngrams(name, 4)) for name in df.name]

In [61]:
stg = TfidfVectorizer(ngram_range=(2,2), analyzer='char')
mat_trg = stg.fit_transform(x_train)

In [62]:
nb = MultinomialNB()
nb.fit(mat_trg, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
labels = nb.predict(stg.transform(x_test))

In [69]:
print(classification_report(y_test, labels))

             precision    recall  f1-score   support

     female       0.73      0.80      0.76       928
       male       0.78      0.70      0.74       928

avg / total       0.75      0.75      0.75      1856



In [ ]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=default_preprocessor)),
    ('tree', DecisionTreeClassifier(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)

##  Часть 3. Нейросетевая 

### Нейронная сеть [5 баллов]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$ – то есть, используется one hot encoding.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

#### делаем one-hot-encoded данные

In [12]:
alphabet = sorted(set(''.join(df['name'])))

In [13]:
maxlen = max(df['name'].map(lambda x: len(x)))
maxlen

15

In [14]:
def one_hot(data, letters, maxlen):
    '''np.array вида (кол-во примеров х бинарная матрица размера (количество букв в алфавите × максимальная длина имени)'''
    res = np.zeros((len(data), maxlen, len(letters)))
    for i in range(len(data)): # по именам
        for l in range(len(data[i])): # по буквам
            res[i][l][letters.index(data[i][l])] = 1
    return res

In [15]:
X_train = one_hot(x_train, alphabet, maxlen)
y_train = np.array(list(map(lambda x: (1, 0) if x=='female' else (0, 1), y_train)))
X_train.shape

(7416, 15, 55)

#### а теперь собственно модели

In [16]:
from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from keras.utils import to_categorical

Using TensorFlow backend.


In [17]:
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.output_shape

(None, 2)

In [19]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train, y_train, batch_size=len(X_train), epochs=1, verbose=0)

In [20]:
X_test = one_hot(x_test, alphabet, maxlen)
y_test = to_categorical(np.array(list(map(lambda x: 0 if x=='female' else 1, y_test))), num_classes=2)
preds = model.predict(X_test, verbose=0)

In [21]:
preds = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds)))

In [22]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, preds))
print(accuracy_score(y_test, preds))

             precision    recall  f1-score   support

          0       0.79      0.82      0.80       928
          1       0.81      0.78      0.79       928

avg / total       0.80      0.80      0.80      1856

0.796875


In [211]:
# другие методы
# добавляем дропаут
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds = model.predict(X_test, verbose=0)
preds = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds))
print(accuracy_score(y_test, preds))

             precision    recall  f1-score   support

          0       0.78      0.81      0.80       928
          1       0.81      0.77      0.79       928

avg / total       0.79      0.79      0.79      1856

0.7914870689655172


In [212]:
# меняем значение дропаута
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds2 = model.predict(X_test, verbose=0)
preds2 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds2)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds2))
print(accuracy_score(y_test, preds2))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       928
          1       0.80      0.80      0.80       928

avg / total       0.80      0.80      0.80      1856

0.8033405172413793


In [214]:
# один LSTM-слой
model = Sequential()
model.add(LSTM(128, return_sequences = False, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds3 = model.predict(X_test, verbose=0)
preds3 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds3)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds3))
print(accuracy_score(y_test, preds3))

             precision    recall  f1-score   support

          0       0.77      0.82      0.79       928
          1       0.80      0.75      0.78       928

avg / total       0.79      0.79      0.79      1856

0.7855603448275862


In [23]:
# меняем количество узлов
model = Sequential()
model.add(LSTM(256, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds4 = model.predict(X_test, verbose=0)
preds4 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds4)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds4))
print(accuracy_score(y_test, preds4))

             precision    recall  f1-score   support

          0       0.80      0.82      0.81       928
          1       0.81      0.79      0.80       928

avg / total       0.80      0.80      0.80      1856

0.8033405172413793


In [24]:
# меняем количество слоёв
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds5 = model.predict(X_test, verbose=0)
preds5 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds5)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds5))
print(accuracy_score(y_test, preds5))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       928
          1       0.80      0.80      0.80       928

avg / total       0.80      0.80      0.80      1856

0.7990301724137931


Итог: ничего не меняется! Только лучше два LSTM-слоя, чем один. 

In [28]:
for i in range(len(y_test)):
    if all(y_test[i] != preds5[i]):
        print(x_test[i], y_test[i])

Elsbeth [1. 0.]
Oreste [1. 0.]
Jonell [1. 0.]
Violet [1. 0.]
Margery [1. 0.]
Shel [1. 0.]
Ingaborg [1. 0.]
Nicol [1. 0.]
Deerdre [1. 0.]
Hali [1. 0.]
Venus [1. 0.]
Jenifer [1. 0.]
Orel [1. 0.]
Rakel [1. 0.]
Deirdre [1. 0.]
Wanda [1. 0.]
Taryn [1. 0.]
Nance [1. 0.]
Merci [1. 0.]
Prudence [1. 0.]
Becky [1. 0.]
Beilul [1. 0.]
Bekki [1. 0.]
Idell [1. 0.]
Ines [1. 0.]
Carrol [1. 0.]
Pier [1. 0.]
Katheryn [1. 0.]
Vere [1. 0.]
Burta [1. 0.]
Gates [1. 0.]
Marjy [1. 0.]
Vicky [1. 0.]
Shandy [1. 0.]
Valma [1. 0.]
Nova [1. 0.]
Raven [1. 0.]
Damaris [1. 0.]
Ame [1. 0.]
Mavis [1. 0.]
Mitzi [1. 0.]
Inger [1. 0.]
Brunhilde [1. 0.]
Milicent [1. 0.]
Mommy [1. 0.]
Deloris [1. 0.]
Sukey [1. 0.]
Farand [1. 0.]
Harri [1. 0.]
Greer [1. 0.]
Francesca [1. 0.]
Steffane [1. 0.]
Kimberly [1. 0.]
Audre [1. 0.]
Ardith [1. 0.]
Midge [1. 0.]
Stormi [1. 0.]
Marijo [1. 0.]
Meghan [1. 0.]
Beatriz [1. 0.]
Betty [1. 0.]
Mignon [1. 0.]
Tiff [1. 0.]
Sharl [1. 0.]
Windy [1. 0.]
Sileas [1. 0.]
Cher [1. 0.]
Trudey [1. 0.]
Sto

##  Часть 4. Сомнительная 

### Визуализация [5 баллов]

Побочным продуктом обучения нейронной сети будут эмбеддинги символов – 26 векторов для всех букв латинского алфавита. Попробуйте использовать разные методы снижения размерности – SVD или TSNE – и изобразить точки, соответствующие разным символам, на плоскости. Получаются ли осмысленные и интерпретируемые кластеры? 